# etl.py test notebook
The goal of this notebook is to test the functions from `etl.py` independently

In [8]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *
from etl import process_song_file,process_data

## 1. Testing `process_song_file`

In [2]:
conn = psycopg2.connect(
        "host=127.0.0.1 dbname=sparkifydb user=student password=student"
    )
cur = conn.cursor()

In [3]:
filepath='data/song_data'

In [4]:
all_files = []
for root, dirs, files in os.walk(filepath):
    files = glob.glob(os.path.join(root, '*.json'))
    for f in files:
        all_files.append(os.path.abspath(f))

In [5]:
all_files[:3]

['/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/song_data/A/A/A/TRAAAVG12903CFA543.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/song_data/A/A/A/TRAAABD128F429CF47.json',
 '/home/gabriel/Documents/Repos/udacity_data_engineering_project1/data/song_data/A/A/A/TRAAAAW128F429D538.json']

In [15]:
df = pd.read_json(all_files[0], lines=True)

# insert song record
song_data = (
    df.loc[0, ['song_id', 'title', 'artist_id', 'year', 'duration']]
    .astype(str)
    .tolist()
)
cur.execute(song_table_insert, song_data)

# insert artist record
artist_data = (
    df.loc[
        0,
        [
            'artist_id',
            'artist_name',
            'artist_location',
            'artist_latitude',
            'artist_longitude',
        ],
    ].values.tolist()
)

In [6]:
process_song_file(cur, all_files[0])

In [9]:
process_data(cur, conn, filepath='data/song_data', func=process_song_file)

71 files found in data/song_data
1/71 files processed.
2/71 files processed.
3/71 files processed.
4/71 files processed.
5/71 files processed.
6/71 files processed.
7/71 files processed.
8/71 files processed.
9/71 files processed.
10/71 files processed.
11/71 files processed.
12/71 files processed.
13/71 files processed.
14/71 files processed.
15/71 files processed.
16/71 files processed.
17/71 files processed.
18/71 files processed.
19/71 files processed.
20/71 files processed.
21/71 files processed.
22/71 files processed.
23/71 files processed.
24/71 files processed.
25/71 files processed.
26/71 files processed.
27/71 files processed.
28/71 files processed.
29/71 files processed.
30/71 files processed.
31/71 files processed.
32/71 files processed.
33/71 files processed.
34/71 files processed.
35/71 files processed.
36/71 files processed.
37/71 files processed.
38/71 files processed.
39/71 files processed.
40/71 files processed.
41/71 files processed.
42/71 files processed.
43/71 file

In [ ]:
process_song_file